##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Temel siniflandirma: Resimdeki kiyafeti tahmin etmek

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Bu kilavuzda bir sinir agi modelini spor ayakkabisi ve gomlek gibi kiyafetlerin resimlerini siiflandirmak icin egitecegiz. Butun detaylari anlamamaniz normal; bu kilavuz detaylari denk geldikce anlatarak butun TensorFlow programini hizli bir sekilde ozetler.

Bu kilavuz yuksek seviye API olan [tf.keras](https://www.tensorflow.org/guide/keras)'i kullanarak TensorFlow icinde modelleri olusturur ve egitir.

In [0]:
try:
  # sadece Colab icinde varolan %tensorflow_version.
  %tensorflow_version 2.x
except Exception:
  pass


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow ve tf.keras
import tensorflow as tf
from tensorflow import keras

# Yardimci kutuphaneler
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Fashion MNIST veri setini iceri aktaralim

Bu kilavuz icinde 10 farkli kategoride 70,000 siyah-beyaz resim bulunduran [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) veri setini kullanir. Asagida gozuktugu gibi dusuk cozunurlukteki (28x28 piksel) bu resimler cesitli kiyafetlere aittir:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST ornekleri</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST makine ogrenmesi programlarinin bir nevi "Hello, World"'u olan klasik [MNIST](http://yann.lecun.com/exdb/mnist/) veri setinin yerine kullanilmistir. MNIST veri seti burada gordugunuz kiyafet resimlerine benzer sekilde bicimlendirilmis, el yazisiyla yazilmis rakamlardan olusur.

Bu kitapcikta Fashion MNIST kullanmamizin sebebi MNIST veri setine gore daha cesitli ve daha zorlayici olmasidir. Iki veri seti de gorece kucuk boyuttadir ve algoritmalarin beklendgini gibi calistigini kontrol etmek icin kullanilir. Ikisi de kodunuzu test etmek ve hatalari duzeltmek icin iyi baslangic noktalaridir.

Burada 60,000 resmi modelimizi egitmek icin, 10,000 resmi de modelin ne kadar dogru tahmin yaptigini degerlendirmek icin kullandik. Fashion MNIST veri setine TensorFlow uzerinden erisebiliriniz. Sadece Fashion MNIST verisini TensorFlow'dan direk olarak iceri aktarin ve yukleyin:

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Veri setinin yuklenmesi dort adet NumPy dizisi dondurur:

* `train_images` ve `train_labels` dizileri *egitim seti* idir—yani modelin ogrenme sirasinda kullanacagi veri.
* Modelin test edilmesi ise *test seti* ile yapilir, `test_images`, ve `test_labels` dizileri ile.

Resimler, piksel degerleri 0 ile 255 arasinda degisen 28x28 NumPy dizileridir. *Etiketler* ise 0 ile 9 arasindaki sayilardan olusan bir dizidir. Bu sayilar her bir kiyafet *sinif* resmini temsil eder:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Her resim bir etiket ile eslestirilmistir. *Sinif isimleri* veri setine dahil edilmedigi icin onlari daha sonra grafik cizerken kullanmak icin burada saklayalim:

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Veriyi inceleyelim

Modelimizi egitmeden once veri setinin yapisini inceleyelim. Asagidaki satirlar egitim setinden 60,000 resim oldugunu ve her resmin 28 x 28 piksel ile temsil edildigini gosterir:

In [0]:
train_images.shape

Ayni sekilde egitim setinde 60,000 etiket bulunmaktadir:

In [0]:
len(train_labels)

Her etiket 0 ile 9 arasinda bir rakamdir:

In [0]:
train_labels

Test setinde is 10,000 resim bulunmaktadir. Yine her resim 28 x 28 piksel ile temsil edilir:

In [0]:
test_images.shape

Ve test seti 10,000 etiket icerir:

In [0]:
len(test_labels)

## Verinin onislenmesi

Modelin egitilmesinden once verinin onislemden gecirilmesi gerekir. Egitim setinin ilk resmini incelediginizde piksel degerlerinin 0 ile 255 arasinda degistigini gorursunuz:

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Bu degerlerin modele verilmeden once 0 ile 1 arasinda bir degere indirgenmesi gerekir. Bunun icin butun degerleri 255 ile bolelim. *Egitim setinin* ve *test setinin* ayni sekilde onislemden gecirilmesi onemlidir:

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

Verilerin dogru yapida oldugundan ve siniur aginin olusturulup egitilmege hazir oldugundan emin olmak icin *egitim setindeki* ilk 25 resmi altlarinda sinif isimleri ile gosterelim.

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Modeli olusturalim

Sinir aglarinin olusturulamasi once model katmanlarinin yapilandirilmasini sonra da bu katmanlarin birlestirilmesini gerektirir.

### Katmanlari kuralim

Sinir aglarinin yapi taslari *katmanlardir*. Katmanlar kendilerine verilen verilerden gosterimleri secip cikartirlar. Bu gosterimlerin elimizdeki sorun icin bir anlam ifade etmesi umit ederiz.

Derin ogrenmenin cogunlugu basit katmanlarin birbirine eklenmesidir. `tf.keras.layers.Dense` gibi bircok katmanin egitim sirasinda ogrendigi degistirgeleri bulunur.

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

Bu agdaki ilk katman, `tf.keras.layers.Flatten`, resim bicimlerini iki-boyutlu diziden (28 x 28 piksel) tek-boyutlu diziye (28 * 28 = 784 piksel) donusturur. Bu katmanin resimdeki piksel siralarini kaldirip yan yana siraladigini hayal edin. Bu katmanin ogrenmesi gereken bir degistirge yoktur, sadece verinin yeniden yapilandirilmasindan sorumludur.

Piksellerin duzlestirilmesinden sonra, ag iki `tf.keras.layers.Dense` katmanindan olusur. Bunlar yogun bagli, ya da tam bagli, sinir katmanlaridir. Ilk `Dense` katmanin 128 dugum (ya da sinir hucresi) bulunur. Ikinci (ve son) katman ise 10-dugumluk *softmax* katman olup toplamlari 1 olan 10 olasilik degeri dizisi dondurur. Her dugum icerisinde inceledigi resmin 10 siniftan hangisine ait oldugunun olasilik degerlerini bulundurur.

### Modeli birlestirelim

Model egitilmege hazir olmadan once birkac ayarinin daha yapilmasi gerekir. Bunlar modelin *birlestirilmesi* sirasinda eklenir:

* *Kayip fonksiyonu* —Bu modelin egitim sirasindaki dogrulunu olcer. Bu fonksiyonu modeli dogru tarafa "yoneltmek" icin olabildigince dusuk tutmak istersiniz.
* *Eniyileyici* —Bu modelin gordugu veriye ve kayip fonksiyonuna gore nasil guncellendigidir.
* *Olcumler* —Egitim ve test basamaklarinin takibinde kullanilir. Bu ornekte *dogruluk*, yani resimlerin hangi oranda dogru olarak siniflandirildigini kullaniyoruz.

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Modeli egitelim

Sinir agi modelinin egitilmesi asagidaki basamaklardan olusur:

1. Egitim verisini modelimize besleyelim. Bu ornekteki egitim verisi `train_images` ve `train_labels` dizilerindedir.
2. Model resimleri etiketlerle bagdastirmayi ogrenir.
3. Modele test veri setinde, ornekteki 'test_images' dizisi, tahminler yapmasini sorun. Tahminlerin `test_labels` dizisindeki etiketlerle eslesip eslesmedigini kontrol edin.

Egitime baslamak icin `model.fit` yontemini cagiralim—modeli egitim verisine "oturttugu" icin boyle isimlendirilmistir:

In [0]:
model.fit(train_images, train_labels, epochs=10)

Model egitildikce katip ve dogruluk degerleri gosterilir. Bu modelimiz yaklasik 0.88 (ya da %88) dogruluga ulasmaktadir.

## Dogrulugu degerlendirelim

Simdi modelimizin test veri setinde nasil calistigina bakalim:

In [0]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('\nTest accuracy:', test_acc)

Gordugunuz gibi test veri setindeki dogruluk egitim veri setindeki dogruluktan biraz daha dusuktur. Iki dogruluk arasindaki bu fark *asiri uyum* ornegidir. Asiri uyum makine ogrenmesi modeli yeni ve daha once gormedigi veri setlerinde egitim setinden daha kotu sonuc verdiginde gorulur.

## Tahminde bulunalim

Modelimiz egitildigine gore onu bazi resimler ustunde tahminde bulunmak icin kullanabiliriz.

In [0]:
predictions = model.predict(test_images)

Burada model test setindeki her resim icin bir etiket tahmin etti. Beraber ilk tahmine bakalim:

In [0]:
predictions[0]

Bir tahmin 10 sayidan olusan bir dizidir. Bu sayilar incelenen resmin ait oldugu kiyafet sinifina dair modelin "guvenidir". Hangi etiketin en yuksek guven degerine sahip oldugunu gorebilirsiniz:

In [0]:
np.argmax(predictions[0])

Gordugunuz gibi modelimiz en yuksek guveni kisa bota yani `class_names[9]` elemanina vermis. Test etiketleri dizisine bakarsak bu siniflandirmanin dogru oldugunu goruruz:

In [0]:
test_labels[0]

Butun sinif tahminlerini gormek icin sonucu grafik ustunde gosterelim.

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

0. resme, tahminlere ve tahmin dizisine bakalim. Dogru etiket tahminleri mavi, yanlis etiket tahminleri kirmizi ile belirtilmistir. Sayilar ise etiket tahmininin yuzdesini belirtir.

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

Birden fazla resmi tahminleri ile beraber cizelim. Modelin guven degeri yuksek olmasina ragmen yanlis olabilecegine dikkat edin.

In [0]:
# Ilk X test resmini, tahmin edilen etiketleri ve dogru etiketleri cizin.
# Dogru tahminler mavi, yanlis tahminler kirmizi renktedir.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

Son olarak, egitilmis modeli tek bir resim ustunde tahmin yapmak icin kullanalim.

In [0]:
# Test veri setinden bir resim alalim.
img = test_images[1]

print(img.shape)

`tf.keras` modelleri orneklerin hepsinde ayni anda *toplu* olarak tahmin yapmak icin eniyilestirilmistir. Bu sebepten dolayi tek bir resim kullanmaniza ragmen onu bir diziye eklemelisiniz:

In [0]:
# Resmi tek elemani oldugu bir topluluga ekleyin.
img = (np.expand_dims(img,0))

print(img.shape)

Simdi bu resim icin dogru etiketi tahmin edin:

In [0]:
predictions_single = model.predict(img)

print(predictions_single)

In [0]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

`model.predict` her dizinin veri toplulugundaki bir resme ait oldugu bir diziler dizisi dondurur. Topluluktaki (tek) resmimizin tahminlerini alin:

In [0]:
np.argmax(predictions_single[0])

Model etiketi 2 olarak tahmin eder.